<a href="https://colab.research.google.com/github/kjh1311990/face/blob/main/Fuzzy_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

PARAMS_M = 2047
PARAMS_N = 512

PARAMS_LOGQ = 11
PARAMS_LOGL = 7

PARAMS_Q = 1 << PARAMS_LOGQ
PARAMS_L = 1 << PARAMS_LOGL

In [ ]:
def EMBLEM_Encode(m, shift):
    encm = ((m << 1) + 1) << (shift - 1);
    return encm.reshape(PARAMS_M, 1);

In [ ]:
def EMBLEM_Decode(m, shift):
    decm = m >> shift;
    return decm.reshape(PARAMS_M, 1);

In [ ]:
A = np.random.randint(low = -PARAMS_L/2, high = PARAMS_L/2+1, size = PARAMS_N * PARAMS_M, dtype = int)

In [ ]:
A.resize(PARAMS_M, PARAMS_N)

In [ ]:
A.shape

(2047, 512)

In [ ]:
b = np.random.randint(low = 0, high = 2, size = PARAMS_N, dtype = int);

In [ ]:
b.resize(PARAMS_N,1)

In [ ]:
np.matmul(A,b).shape

(2047, 1)

In [ ]:
#generate message
m = np.random.randint(low = 0, high = 2, size = 133, dtype = int);

In [ ]:
import komm
import numpy as np
code = komm.BCHCode(11,365)

In [ ]:
encm = code.encode(m)

In [ ]:
decm = code.decode(encm)

In [ ]:
encm.size

2047

In [ ]:
EMBLEM_Encode(encm, 10).shape

(2047, 1)

In [ ]:
np.remainder(np.matmul(A,b) + EMBLEM_Encode(encm, 10), PARAMS_Q)

array([[ 707],
       [ 228],
       [1768],
       ...,
       [1708],
       [1294],
       [1880]], dtype=int32)